# Insight Project --Birding Big Year--

In this project I intend to determine a way to see all the birds one can see on a single state, for a given time window.  For all those birdirers that want to get to the top 100 of their state on ebrid, this will be the perfect tool. The user will input the state, home address (or lat,lon), time window and birds that already have been seen*. This last one (*) is an optional thing.

In [ ]:
import numpy as np
from datetime import datetime
import geopandas as gpd
from shapely.geometry import Point
import os
import struct
import pickle
import googlemaps

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib
from mpl_toolkits.axes_grid1 import make_axes_locatable

from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN

import pandas as pd
from pandas.io.json import json_normalize, read_json

def save_fig(name):
    fig.savefig(name,dpi=80,bbox_inches='tight', pad_inches=0.02, format = 'png')

%matplotlib inline

# The ebird Data

I will start with a singe state. Since the ebird API limits the type of request I can make, I have a downloaded the cvs file.  I'm using the last two full years of data but in reality the alorithm should be train with more data and just tested on the last year.

In [ ]:
# dfAll = pd.read_csv('./ebd_US-WY_201801_201912_relApr-2020/ebd_US-WY_201801_201912_relApr-2020.txt'
#                 ,delimiter="\t")

dfAll = pd.read_csv('./ebd_US-WI_201801_201912_relApr-2020/ebd_US-WI_201801_201912_relApr-2020.txt'
                ,delimiter="\t", usecols=['CATEGORY', 'LOCALITY TYPE', 'ALL SPECIES REPORTED', 'APPROVED',
                                         'SAMPLING EVENT IDENTIFIER', 'COMMON NAME', 'LOCALITY', 
                                          'LATITUDE', 'LONGITUDE',
                                          'OBSERVATION DATE', 'ALL SPECIES REPORTED'])

# dfAll = pd.read_csv('./ebd_US-WI_201001_201812_relApr-2020/ebd_US-WI_201001_201812_relApr-2020.txt'
#                 ,delimiter="\t")

I add sertain condition to satify completnes fo the data, public locations and only bird species (i.e. no hybirds). `dfReduce` will contian all the information I will be using.

In [ ]:
dfAll = dfAll[(dfAll['CATEGORY'] == 'species') & (dfAll['LOCALITY TYPE'] == 'H')
              & (dfAll['ALL SPECIES REPORTED'] == 1)  & (dfAll['APPROVED'] == 1)]

In [ ]:
dfReduce = dfAll.filter(['SAMPLING EVENT IDENTIFIER', 'COMMON NAME', 'LOCALITY',
              'LATITUDE', 'LONGITUDE', 'OBSERVATION DATE', 'ALL SPECIES REPORTED']) 
dfReduce['OBSERVATION DATE'] = pd.to_datetime(dfReduce['OBSERVATION DATE'])
dfReduce['YEAR WEEK'] = dfReduce['OBSERVATION DATE'].dt.strftime('%W')
dfReduce['YEAR DAY'] = dfReduce['OBSERVATION DATE'].dt.strftime('%j')
dfReduce['YEAR'] = dfReduce['OBSERVATION DATE'].dt.strftime('%Y')
dfReduce['YEAR WEEK'] = pd.to_numeric(dfReduce['YEAR WEEK'])

In [ ]:
dfReduce.head(5)

dfReduce contains both my train set and my validation set.  In this case I will use the last year as my validation set (2019) and all the previous information as my train set.

In [ ]:
dfValidation = dfReduce[dfReduce['YEAR']==2019]

In [ ]:
dfTrain = dfReduce[dfReduce['YEAR']!=2019]

In [ ]:
del dfReduce
del dfAll

In [ ]:
len(np.unique(dfTrain['LOCALITY'].values))

# Clustering using BDSCAN

BDSCAN is a density clustering that will tell where is popular for people to go birding (based on the desnity of hotsopts).  I will define a cluster as having atleast 3 point and with a maximum distance of 0.05degrees or about 5km.  With that I will optain where does each hotspot ('LOCALITY') belongs to. If '-1' they are not part of any cluster.

In [ ]:
dfbdscan = dfTrain.filter(['LOCALITY','LATITUDE', 'LONGITUDE'])

In [ ]:
dfbdscan.drop_duplicates(subset='LATITUDE', keep = 'first', inplace = True)

In [ ]:
dfbdscan.head(5)

In [ ]:
latHotspot, lonHotspot = np.array(dfbdscan['LATITUDE']), np.array(dfbdscan['LONGITUDE'])
locationList = np.empty((latHotspot.shape[0],2))
for i in range(0, dfbdscan.shape[0]):
    locationList[i,0], locationList[i,1] = latHotspot[i], lonHotspot[i]
    

In [ ]:
eps = .05
dbmod = DBSCAN(eps=eps, min_samples=3, metric='euclidean').fit(locationList)

In [ ]:
dbmod.components_

In [ ]:
pickle.dump(dbmod, open("./bdclusterModel.p", "wb" ))

In [ ]:
labels = dbmod.labels_
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)
core_samples_mask = np.zeros_like(dbmod.labels_, dtype=bool)
core_samples_mask[dbmod.core_sample_indices_] = True

print('Estimated number of clusters: %d' % n_clusters_)
print('Estimated number of noise points: %d' % n_noise_)


In [ ]:
unique_labels = set(labels)
colors = [plt.cm.viridis_r(each)
          for each in np.linspace(0, 1, len(unique_labels))]


In [ ]:
for i, clust in enumerate(labels):
    plt.scatter(locationList[i][1],locationList[i][0], color = colors[clust])
plt.show()

In [ ]:
dfbdscan['BD CLUSTER'] = dbmod.labels_

In [ ]:
dfbdscan.head(5)

In [ ]:
def hotspot_finder_for_dbcluster(k, df, dfbdscan):
    dfIntermediate = dfTrain[dfTrain['LOCALITY'].isin(list(dfbdscan[dfbdscan['BD CLUSTER'] == k]['LOCALITY']))].groupby(['LOCALITY','COMMON NAME']).sum().filter(['ALL SPECIES REPORTED'])
    dfIntermediate['BIRD'] = list(map(lambda x: 1 if x < 1e6 else 1, dfIntermediate['ALL SPECIES REPORTED']))
    location  = dfIntermediate.filter(['BIRD']).reset_index().groupby(['LOCALITY']).sum().reset_index().sort_values(by='BIRD', ascending=False).reset_index()['LOCALITY'][0]
    lat = df[df['LOCALITY']==location].reset_index()['LATITUDE'][0]
    lon = df[df['LOCALITY']==location].reset_index()['LONGITUDE'][0]
    
    return location, lat, lon

In [ ]:
coorHotspot = np.empty((n_clusters_, 2))
for i in range(0, n_clusters_):
    a, coorHotspot[i, 0], coorHotspot[i, 1] = hotspot_finder_for_dbcluster(i, dfTrain, dfbdscan)

### Now some good plots

In [ ]:
country = gpd.read_file('/Users/casanova/DocumentsHere/Insight/gz_2010_us_040_00_5m.json')

In [ ]:
# state = 'Wyoming'
state = 'Wisconsin'

In [ ]:
fig, ax = plt.subplots(1, figsize=(7,8))
base = country[country['NAME'].isin([state]) == True].plot(ax=ax, color='#3B3C6E', alpha = 0.3)
for i, clust in enumerate(labels):
    ax.scatter(locationList[i][1],locationList[i][0], color = colors[clust])
ax.scatter(coorHotspot[:,1],coorHotspot[:,0], marker = 'x', color = 'r', s=80)
ax.set_ylabel(r'Latitude [$^o$]')
ax.set_xlabel(r'Longitude [$^o$]')

plt.show()
save_fig('/Users/casanova/DocumentsHere/Insight/{}-plain.png'.format(state))

#### Now the bird probability.

`dfbdscan` have the information of where each of the hotspots lay, in terms of their cluster.  Now in order to constuct a path is important to mask the probabilites of the of seeing a particular bird with T or F on a weekly basis.  This is critical in order to construc the sets.

In [ ]:
dfProb = dfTrain.merge(dfbdscan.filter(['LOCALITY','BD CLUSTER']),
                            left_on='LOCALITY', right_on='LOCALITY', how = 'left').filter(['COMMON NAME','ALL SPECIES REPORTED','YEAR WEEK', 'BD CLUSTER'])

In [ ]:
dfProb.head(5)

In [ ]:
nTime = 54
nLoc = n_clusters_
setMat = np.empty((nTime,nLoc), dtype=object)

In [ ]:
for week in range(0,nTime):
    dfProbA = dfProb[dfProb['YEAR WEEK']== week]
    dfProb1 = dfProbA.groupby(['COMMON NAME','BD CLUSTER']).sum().filter(['ALL SPECIES REPORTED']).reset_index()
    dfProb1.rename(columns = {'ALL SPECIES REPORTED':'POS OBS'}, inplace=True)
    dfProb2 = dfProbA.groupby(['BD CLUSTER']).sum().filter(['ALL SPECIES REPORTED']).reset_index()
    dfProb2.rename(columns = {'ALL SPECIES REPORTED':'TOT OBS'}, inplace=True)
    dfProb3 = dfProb1.merge(dfProb2, left_on='BD CLUSTER', right_on='BD CLUSTER', how = 'left')
    dfProb3['POS PROB'] = dfProb3['POS OBS']/dfProb3['TOT OBS']
    for loc in range(0,nLoc):
        dfWeek = dfProb3[dfProb3['BD CLUSTER'] == loc]
        dfWeek['TF aa'] = list(map(lambda x: 0 if x < 0.02 else 1, dfWeek['POS PROB']))
        setMat[week,loc] = set(dfWeek[dfWeek['TF aa'] == 1]['COMMON NAME'].values)
        

In [ ]:
pickle.dump(setMat, open("./2dSetLocations.p", "wb" ))

In [ ]:
ToMakeUniverse = list(setMat.flatten())
Universe = set(e for s in ToMakeUniverse for e in s)

In [ ]:
list(Universe)

# Here we go!!!!!

First user inputs some coordinates.
Then the coordinates get translated to a cluster.
That give us the first set (first week)
Then we obtain the rest of the sets. The key here is to back track a set to an actual 'x,t' entry so we can have a route.
Display in some way that list of locations!  (Probabily using the centroid maps or coordinates).

In [ ]:
# userInputLat,userInputLon = 44, -110
userInputLat,userInputLon = 43.069511, -89.396723

userInput = [userInputLat,userInputLon]
print(userInput)

On the first week I most see:

In [ ]:
kmeansLoaded = pickle.load(open("./bdclusterModel.p", "rb" ))
setMatLoaded = pickle.load(open("./2dSetLocations.p", "rb" ))

In [ ]:
# print('you start in location:',kmeansLoaded.predict([userInput])[0])
# initialLocSet = setMat[0,kmeansLoaded.predict([userInput])[0]]
# print(list(initialLocSet))

The hole list of bird that we are planing to see are:

In [ ]:
ToMakeUniverse = list(setMatLoaded.flatten())
Universe = set(e for s in ToMakeUniverse for e in s)

In [ ]:
print('With a total of', len(list(Universe)), 'birds')

In [ ]:
def set_cover_mine(elements, subsets):
    '''
    There is a greedy algorithm for polynomial time approximation of set covering that chooses sets according to one rule: at each stage, choose the set that contains the largest number of uncovered elements.

    
    '''
    covered = set() 
    cover = []
    listCover = []
    # Greedily add the subsets with the most uncovered points
    while covered != elements:
        subset = max(subsets, key=lambda s: len(s - covered))
        cover.append(subset)
        listCover.append(subsets.index(subset))
        covered |= subset
 
    return cover, listCover





In [ ]:
setList, locList = set_cover_mine(Universe, ToMakeUniverse)


In [ ]:
locList = np.sort(locList)

In [ ]:
nTime, nLoc = setMatLoaded.shape
locMat = np.linspace(1,nTime*nLoc,nTime*nLoc).reshape(nTime,nLoc)

In [ ]:
outList = []
for element in locList:
    a,b = np.where(locMat == element)
    outList.append('On week {}, you need to be at location {}'.format(a[0],b[0]))
    

In [ ]:
outList

# Playing with the data

In [15]:
import SetCover
import numpy as np

In [7]:
S = [set([1,2]), 
     set([1]), 
     set([1,2,3]), 
     set([1]), 
     set([3,4]), 
     set([4]), 
     set([1,2]), 
     set([3,4]), 
     set([1,2,4])]

C = [1, 1, 2, 2, 2, 3, 3, 4, 4]

U = set([1, 2, 3, 4])

In [8]:
SetCover.universe_maker(S)

{1, 2, 3, 4}

In [9]:
SetCover.set_cover_greedy(U,S)

([{1, 2, 3}, {3, 4}], [2, 4])

In [10]:
max(S, key=lambda s: len(s - set([1])))

{1, 2, 3}

In [11]:
C0 = C/np.mean(C)
a = list(map(lambda x, y: len(x -set([1]))/y if len(x -set([1])) != 0 else 1e6, S, C0))
print(a)
print(a.index(min(a)))
print(S[a.index(min(a))])

[2.4444444444444446, 1000000.0, 2.4444444444444446, 1000000.0, 2.4444444444444446, 0.814814814814815, 0.814814814814815, 1.2222222222222223, 1.2222222222222223]
5
{4}


In [16]:
SetCover.set_cover_weighted_greedy(U,S,C)

NameError: name 'np' is not defined

# Plaing with google distances

In [ ]:
gmaps = googlemaps.Client(key='')

In [ ]:
geocode_result = gmaps.geocode('1600 Amphitheatre Parkway, Mountain View, CA')

In [ ]:
geocode_result

In [ ]:
reverse_geocode_result = gmaps.reverse_geocode((40.714224, -73.961452))

In [ ]:
reverse_geocode_result

In [ ]:
distanceMatGmaps = gmaps.distance_matrix(origins = locationList, 
                                         destinations=locationList, 
                                         mode = 'driving', 
                                         units = 'metric')

In [ ]:
b = gmaps.distance_matrix(origins = [(latHotspot[0], lonHotspot[0]),
                                       (latHotspot[1], lonHotspot[1]),
                                       (latHotspot[2], lonHotspot[2]),
                                       (latHotspot[3], lonHotspot[3]),
                                       (latHotspot[4], lonHotspot[4]),
                                       (latHotspot[5], lonHotspot[5]),
                                       (latHotspot[6], lonHotspot[6]),
                                       (latHotspot[7], lonHotspot[7]),
                                       (latHotspot[8], lonHotspot[8]),
                                       (latHotspot[9], lonHotspot[9])],
                          destinations=[(latHotspot[0], lonHotspot[0]),
                                       (latHotspot[1], lonHotspot[1]),
                                       (latHotspot[2], lonHotspot[2]),
                                       (latHotspot[3], lonHotspot[3]),
                                       (latHotspot[4], lonHotspot[4]),
                                       (latHotspot[5], lonHotspot[5]),
                                       (latHotspot[6], lonHotspot[6]),
                                       (latHotspot[7], lonHotspot[7]),
                                       (latHotspot[8], lonHotspot[8]),
                                       (latHotspot[9], lonHotspot[9])],
                         mode = 'driving', units = 'metric')

In [ ]:
b

In [ ]:
b['rows']

In [ ]:
b['rows'][1]['elements'][2]['duration']['value']

In [ ]:
distMat = np.empty((10,10))

for i in range(0,10):
    for j in range(0,10):
        distMat[i,j] = b['rows'][i]['elements'][j]['duration']['value']/3600

In [ ]:
print(distMat)

In [ ]:
15391/3600